In [7]:
import tweepy
import datetime 
from twitter_authentication import bearer_token
import time
import pandas as pd
import os
import logging

In [3]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [4]:
if os.path.isdir('data') == False:
    os.mkdir('data')

In [ ]:
if os.path.isdir('logs') == False:
    os.mkdir('logs')

In [12]:
logging.basicConfig(filename="./logs/std.log", 
					format='%(asctime)s %(message)s', 
					filemode='a') 

In [11]:
logger=logging.getLogger()
logger.setLevel(logging.DEBUG)

In [13]:
logger.debug("This is just a harmless debug message") 
logger.info("This is just an information for you") 
logger.warning("OOPS!!!Its a Warning") 
logger.error("Have you try to divide a number by zero") 
logger.critical("The Internet is not working....") 

### BTC Train Set 2017 to 2020

In [5]:
asset_name = "BTC"

In [ ]:
date = datetime.datetime(2016,12,31,23,0,0)
date_list = []
for i in range(0, 3 * 365 * 24 + 1):
    date += datetime.timedelta(hours=1)
    date_str = date.strftime("%Y-%m-%dT%H:%M:%SZ")
    date_list.append(date_str)

In [ ]:
print (date_list)

In [ ]:
tweets_list = []

count = 0

for j in range(0, len(date_list) - 1):
    
    count = count + 1
    
    logger.info("") 
    logger.info(date_list[j])
    logger.info(date_list[j+1])

    for response in tweepy.Paginator(client.search_all_tweets, 
                                    query = '#btc OR #bitcoin OR #bitcointrading -is:retweet lang:en',
                                    user_fields = ['username', 'public_metrics', 'description', 'location'],
                                    tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                    expansions = 'author_id',
                                    start_time = date_list [j],
                                    end_time = date_list [j+1],
                                    max_results=10,  
                                    limit=25):
        
        time.sleep(1)
        tweets_list.append(response)
        
    
    if count % 24 == 0:
        
        logger.info("output to csv")
        
        path = './data/'+ asset_name + '__' + date_list[j][0:10] + '.csv'
        
        result = []
        user_dict = {}
        
        # Loop through each response object
        for response in tweets_list:
            # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
            for user in response.includes['users']:
                user_dict[user.id] = {'username': user.username, 
                                    'followers': user.public_metrics['followers_count'],
                                    'tweets': user.public_metrics['tweet_count'],
                                    'description': user.description,
                                    'location': user.location
                                    }
            for tweet in response.data:
                # For each tweet, find the author's information
                author_info = user_dict[tweet.author_id]
                # Put all of the information we want to keep in a single dictionary for each tweet
                result.append({'author_id': tweet.author_id, 
                            'username': author_info['username'],
                            'author_followers': author_info['followers'],
                            'author_tweets': author_info['tweets'],
                            'author_description': author_info['description'],
                            'author_location': author_info['location'],
                            'text': tweet.text,
                            'created_at': tweet.created_at,
                            'retweets': tweet.public_metrics['retweet_count'],
                            'replies': tweet.public_metrics['reply_count'],
                            'likes': tweet.public_metrics['like_count'],
                            'quote_count': tweet.public_metrics['quote_count']
                            })

        # Change this list of dictionaries into a dataframe
        df = pd.DataFrame(result)
        
        df.to_csv(path)
        
        tweets_list = []
        
        
        